In [0]:
import gym
import numpy as np
import random
import time
from IPython.display import clear_output

In [0]:
env = gym.make("FrozenLake-v0")
# NOTE: default ice is slippery:
# def __init__(self, desc=None, map_name="4x4",is_slippery=True):

In [163]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n
#LEFT = 0 DOWN = 1 RIGHT = 2 UP = 3

q_table = np.zeros((state_space_size, action_space_size))

print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [0]:
# @hyperparameters
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

# Exploration parameters
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [0]:
rewards_all_episodes = []

# Q-learning algorithm
for episode in range(num_episodes):
    # initialize new episode params
    state = env.reset()
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        ## If this number > greater than epsilon --> exploitation 
        #(taking the biggest Q value for this state)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)
        # print(f"new_state: {new_state}, reward: {reward}, done: {done}, info: {info}")

        # Update Q-table for Q(s,a)
        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        # Set new state
        state = new_state
        # Add new reward
        rewards_current_episode += reward 
        # check if the last action ended the episode
        if done == True: 
            break

    # Exploration rate decay
    # Reduce epsilon (because we need less and less exploration)
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)

    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)

    # move on to the next episode

In [166]:
# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.05000000000000004
2000 :  0.19200000000000014
3000 :  0.3980000000000003
4000 :  0.5310000000000004
5000 :  0.6410000000000005
6000 :  0.6730000000000005
7000 :  0.6400000000000005
8000 :  0.6600000000000005
9000 :  0.6830000000000005
10000 :  0.6700000000000005


In [167]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)



********Q-table********

[[0.51467849 0.49774315 0.50149292 0.50244804]
 [0.37764892 0.2939167  0.28203242 0.48070561]
 [0.40576112 0.40174314 0.41165077 0.45112749]
 [0.20403564 0.33891561 0.28212005 0.44056219]
 [0.52732514 0.36686744 0.35745469 0.41978229]
 [0.         0.         0.         0.        ]
 [0.16461825 0.12622172 0.32100439 0.13242672]
 [0.         0.         0.         0.        ]
 [0.22276788 0.28482594 0.39879923 0.55446893]
 [0.40467932 0.61287852 0.51682563 0.37682515]
 [0.50969654 0.31190255 0.29604709 0.22376167]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.4241938  0.53024321 0.76558472 0.5401308 ]
 [0.76691883 0.87905586 0.70873493 0.76762036]
 [0.         0.         0.         0.        ]]


In [168]:
# Watch our agent play Frozen Lake by playing the best action 
# from each state according to the Q-table

# the (Left) or (Down) is indicating the Last action
# if self.lastaction is not None:
#   outfile.write("  ({})\n".format(["Left","Down","Right","Up"][self.lastaction]))

for episode in range(3):
    # initialize new episode params
    state = env.reset()
    done = False
    print("\n*****EPISODE ", episode+1, "*****\n")
    # time.sleep(1)

    for step in range(max_steps_per_episode):
        # clear_output(wait=True)
        # Show current state of environment on screen
        env.render()
        # time.sleep(0.3) 

        # Choose action with highest Q-value for current state
        action = np.argmax(q_table[state,:])
        # Take new action
        new_state, reward, done, info = env.step(action)
        
        if done:
            # clear_output(wait=True)
            # env.render()
            if reward == 1:
                # Agent reached the goal and won episode
                print("****You reached the goal!****")
                # time.sleep(3)
            else:
                # Agent stepped in a hole and lost episode
                print("****You fell through a hole!****")
                # time.sleep(3)
            # clear_output(wait=True)
            break
            
        # if not done, set new state
        state = new_state
        
env.close()


*****EPISODE  1 *****


SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
****You reached the goal!****

*****EPISODE  2 *****


SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFF